In [1]:
import pandas as pd
import numpy as np
import copy as cp
import darc_core
import pickle
import sys
import hashlib as hs
import base64 as b64
import os
import random as rd
import datetime as dt
from darc_core.metrics import Metrics
from darc_core.utils import M_COL, T_COL, T_COL_IT, NB_GUESS, SIZE_POOL
from darc_core.preprocessing import round1_preprocessing
from darc_core.utils import check_format_trans_file

In [14]:
th = pd.read_csv("./S_tahi_sub_2.csv", parse_dates=["date"])
th=th.sort_values("date")
th["month"] = th["date"].dt.month
th["year"]=th["date"].dt.year

In [17]:
gt

,id_user,date,hours,id_item,price,qty
0,17850,2010-12-01,08:26,85123A,2.55,6
1,17850,2010-12-01,08:26,71053,3.39,6
2,17850,2010-12-01,08:26,84406B,2.75,8
3,17850,2010-12-01,08:26,84029G,3.39,6
4,17850,2010-12-01,08:26,84029E,3.39,6
5,17850,2010-12-01,08:26,22752,7.65,2
6,17850,2010-12-01,08:26,21730,4.25,6
7,17850,2010-12-01,08:28,22633,1.85,6
8,17850,2010-12-01,08:28,22632,1.85,6
9,13047,2010-12-01,08:34,84879,1.69,32


In [5]:
gt = pd.read_csv("./ground_truth.csv", parse_dates=["date"])

In [6]:
gt

,id_user,date,hours,id_item,price,qty
0,17850,2010-12-01,08:26,85123A,2.55,6
1,17850,2010-12-01,08:26,71053,3.39,6
2,17850,2010-12-01,08:26,84406B,2.75,8
3,17850,2010-12-01,08:26,84029G,3.39,6
4,17850,2010-12-01,08:26,84029E,3.39,6
5,17850,2010-12-01,08:26,22752,7.65,2
6,17850,2010-12-01,08:26,21730,4.25,6
7,17850,2010-12-01,08:28,22633,1.85,6
8,17850,2010-12-01,08:28,22632,1.85,6
9,13047,2010-12-01,08:34,84879,1.69,32
